# Imports

In [35]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
from google.colab import files

# Carga de los data sets de test y de entrenamiento

In [6]:
test = pd.read_csv('https://raw.githubusercontent.com/FerFabbiano/tp2-organizacionDeDatos/main/Test_TP2_Datos_2020-2C.csv')
train = pd.read_csv('https://raw.githubusercontent.com/FerFabbiano/tp2-organizacionDeDatos/main/Train_TP2_Datos_2020-2C.csv')

trainSetProcesado = pd.read_csv('https://raw.githubusercontent.com/FerFabbiano/tp2-organizacionDeDatos/main/train_dataset_procesado.csv')
testSetProcesado = pd.read_csv('https://raw.githubusercontent.com/FerFabbiano/tp2-organizacionDeDatos/main/test_set_procesado.csv')

train.drop_duplicates(subset="Opportunity_ID", inplace=True)
test.drop_duplicates(subset="Opportunity_ID", inplace=True)

Separamos la variable a predecir. Esta vez vamos a utilizar el data frame procesado, que contiene el feature "Total Taxable Amount USD".

In [7]:
#X = DATA , Y = TARGET 
X1, y1 = trainSetProcesado[['Bureaucratic_Code_0_Approval', 'Total_Taxable_Amount_USD']], trainSetProcesado['Stage']

Dividimos el set de entrenamiento en train y test, utilizando una proporción del 30% del total de los datos para el segundo.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.3, random_state=45)

# Stacking Classifier

In [57]:
estimators = [
    ('rf', RandomForestClassifier()),
    ('xgb', XGBClassifier())
]

clf = StackingClassifier(
    estimators = estimators, final_estimator = LGBMClassifier()
)

clf.fit(X_train, y_train)

StackingClassifier(cv=None,
                   estimators=[('rf',
                                RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       max_samples=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                     

In [59]:
predicciones_train = clf.predict_proba(X_test)

In [60]:
from sklearn.metrics import log_loss
log_loss(y_test, predicciones_train)

0.4344481826249795

In [54]:
testSetProcesado.drop_duplicates(subset="Opportunity_ID", inplace=True)

In [55]:
predicciones_finales = clf.predict_proba(testSetProcesado[['Bureaucratic_Code_0_Approval','Total_Taxable_Amount_USD']])

In [56]:
predicciones_exito = []
for prob in predicciones_finales:
  predicciones_exito.append(prob[1])

In [33]:
csv_kaggle = pd.DataFrame()
csv_kaggle['Opportunity_ID'] = testSetProcesado["Opportunity_ID"]
csv_kaggle['target'] = predicciones_exito

In [36]:
csv = csv_kaggle.to_csv('predicciones_stacking_classifier.csv', index=False)
files.download('predicciones_stacking_classifier.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Resultados

Algoritmo: **Ensemble - Stacking Classifier**

**XgBoost + Random Forest --> Light GBM**

Features analizados: 

- Total Taxable Amount
- Bureaucrotic Code 0 Approval

**Resultado Kaggle = 0,72560**

Nuevamente tenemos un caso de overfitting ya que los resultados de Kaggle son superiores a los esperados.